In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
from IPython.display import display
from tensorflow.keras.layers import Conv2D, Dropout, Flatten
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow import keras 
import math
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [3]:
import warnings
warnings.filterwarnings("ignore")
tf.get_logger().setLevel(tf.compat.v1.logging.ERROR)

In [4]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [5]:
import tensorflow.keras.backend as KTF
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   # Dynamic allocation
sess = tf.compat.v1.Session(config=config)
#KTF.set_session(sess)
path = './GPR_npy/npy_train_data'

## 載入資料

In [6]:
real_data = np.load(path+'/RSSI_train.npy', allow_pickle=True)
AP = len(real_data[0])
AP

158

## 定義 loss function

In [8]:
def vae_loss_function(reconstruction, x,rssi_gpr, mu, log_var):
    x_temp = tf.reshape(x, [-1, AP])
#     q = np.quantile(x_temp.numpy(),0.05)
    
    x_idx = x_temp != -100
    x_idx_h = x_temp >= -55
    
    gt = x_temp.numpy().copy()
    gt[gt <= -100] = np.nan
    mean_gt = np.nanmean(gt, axis=1)
#     max_gt_idx = np.nanmax(gt, axis=1) > -60
    var_gt = np.nanvar(gt, axis=1)
    
    rec = reconstruction.numpy().copy()
    rec[rec <= -100] = np.nan
    mean_rec = np.nanmean(rec, axis=1)
    var_rec = np.nanvar(rec, axis=1)
    
    mv = []
    mean_loss = tf.reduce_sum(abs(mean_gt-mean_rec))
    var_loss = tf.reduce_sum(abs(var_gt-var_rec))
    mv.append(tf.reduce_sum(mean_loss + var_loss))
    mv = tf.reduce_sum(mv)
    
#     mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
#     mse_loss = mse(x_temp[x_idx], reconstruction[x_idx])
#     reconstruction_loss = tf.reduce_sum(mse_loss)
    
    l1 = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    l1_loss = l1(x_temp[x_idx], reconstruction[x_idx]) 
    reconstruction_loss_l1 = tf.reduce_sum(l1_loss)
    
    l1 = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    l1_loss = l1(x_temp, reconstruction) 
    l1_loss = abs(x_temp - 100) * abs(x_temp - 100) * l1_loss
    reconstruction_loss_l1_100 = tf.reduce_sum(l1_loss)
    
    l1 = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    x_idx_max = x_temp == np.nanmax(x_temp)
    x_idx_min = x_temp == np.nanmin(x_temp)    
    l1_loss_max = l1(x_temp[x_idx_max], reconstruction[x_idx_max])/np.sum(x_idx_max)
    l1_loss_min = l1(x_temp[x_idx_min], reconstruction[x_idx_min])/np.sum(x_idx_min)
#     print(np.sum(x_idx_max), np.sum(x_idx_min))
    reconstruction_loss_l1_mm = l1_loss_max + l1_loss_min
    
    
    
    l1 = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    l1_loss_h = l1(x_temp[x_idx_h], reconstruction[x_idx_h])
    reconstruction_loss_l1_h = tf.reduce_sum(l1_loss_h)
    
    kl_loss = -0.5 * (1 + log_var - tf.square(mu) - tf.exp(log_var))
    kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
    
    li = [reconstruction_loss_l1_100 * 1.2/10000000 , reconstruction_loss_l1_mm * 100, mv * 0.01, kl_loss]

#     flag = 0
    total_loss = reconstruction_loss_l1
    for i in li:
        if not math.isnan(i):
            if i != 0:
                total_loss += i
    return total_loss,reconstruction_loss_l1,reconstruction_loss_l1_h,mean_loss,var_loss,kl_loss

## 定義 model

In [9]:
class VAE(Model):
    def __init__(self,z_dim,initializer,**kwargs):
        super(VAE, self).__init__(**kwargs)
        self.leakyReLU = layers.LeakyReLU()
        self.ReLU = layers.ReLU()
#         self.dropout = tf.keras.layers.Dropout(.05)
        
        self.f1 = layers.Dense(AP*3,input_shape=(AP,),kernel_initializer=initializer)
        self.f2 = layers.Dense(int(AP*2.8),kernel_initializer=initializer)
        self.f3 = layers.Dense(int(AP*2.5),input_shape=(AP,),kernel_initializer=initializer)
        self.f4 = layers.Dense(int(AP*2),kernel_initializer=initializer)
        self.fc11 = layers.Dense(int(AP*1.8),kernel_initializer=initializer)
        self.fc12 = layers.Dense(z_dim,kernel_initializer=initializer)

        self.fc21 = layers.Dense(int(AP*1.8),kernel_initializer=initializer)
        self.fc22 = layers.Dense(z_dim,kernel_initializer=initializer)

        # For decoder
        self.fc5 = layers.Dense(int(AP*2.5),kernel_initializer=initializer)
        self.fc6 = layers.Dense(int(AP*2),kernel_initializer=initializer)
        self.fc7 = layers.Dense(int(AP*1.8),kernel_initializer=initializer)
        self.fc8 = layers.Dense(int(AP*1.5),kernel_initializer=initializer)
        self.fc9 = layers.Dense(AP,kernel_initializer=initializer)
    
    def encoder(self, x):
        x = tf.reshape(x, [-1,AP])
        h = self.leakyReLU(self.f1(x))
#         h = self.dropout(h)
        h = self.leakyReLU(self.f2(h))
        h = self.leakyReLU(self.f3(h))
        h = self.f4(h)
        mu = self.leakyReLU(self.fc11(h))
        mu = self.fc12(mu)

        log_var = self.leakyReLU(self.fc21(h))
        log_var = self.fc22(log_var)
        return mu, log_var  # mu, log_var
    
    def decoder(self, z):
        h = self.leakyReLU(self.fc5(z))
        h = self.leakyReLU(self.fc6(h))
        h = self.leakyReLU(self.fc7(h))
        h = self.leakyReLU(self.fc8(h))
        h = self.fc9(h)
        return h 
    
    def sampling(self, mu, log_var):
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon
    
    def call(self, x):
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        reconstruction = self.decoder(z)
        return reconstruction, mu, log_var, z

## 設置 train dataloader

In [10]:
pos_data = np.load(path+'/pos_train.npy', allow_pickle=True)
real_data = np.load(path+'/RSSI_train.npy', allow_pickle=True)
inter_data = np.load(path+'/RSSI_pred_train.npy', allow_pickle=True)
BUFFER_SIZE = real_data.shape[0]
# keep_threshold = 1

def train_generator():
    li = []
    length = len(real_data)
    for index in range(length):
        real_fl_array = np.array(real_data[index], dtype=np.float32).reshape(AP,1)
        inter_fl_array = np.array(inter_data[index], dtype=np.float32).reshape(AP,1)
        pos_f1_array = np.array(pos_data[index], dtype=np.float32)
        yield inter_fl_array, real_fl_array, pos_f1_array

In [11]:
batch_size = 100
train_ds = tf.data.Dataset.from_generator(
    train_generator,
    output_shapes=((AP,1),(AP,1),(2,)), 
    output_types=(tf.float32,tf.float32,tf.float32))

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.batch(batch_size).cache().prefetch(buffer_size=AUTOTUNE).shuffle(batch_size).repeat(3)

## 設置訓練函式

In [12]:
def train_step(rssi_gpr, rssi_gt, pos_gt,train=False):
    
    with tf.GradientTape() as vae_tape, tf.GradientTape() as pos_tape:
        reconstruction, mu, log_var, z = vae(rssi_gpr, training=train)
        
        vae_loss, vae_l1_loss, vae_l1_loss_h,mean_loss,var_loss,kl_loss = vae_loss_function(reconstruction,rssi_gt,rssi_gpr, mu, log_var)
    if not math.isnan(vae_loss):
        gradients_of_vae = vae_tape.gradient(vae_loss, vae.trainable_variables)
        vae_optimizer.apply_gradients(zip(gradients_of_vae, vae.trainable_variables))

    return vae_loss, vae_l1_loss, vae_l1_loss_h,mean_loss,var_loss,kl_loss

In [13]:
def trainer(start_ep,epochs):
    min_l1 = 999
    skip = 0
    flag = 0
    start_time = time.time()
    
    for epoch in range(start_ep,epochs):
        vae_loss_li = []
        pos_loss_li = []
        vae_loss_li_l1 = []
        mloss_li = []
        vloss_li = []
        kl_li = []
#         mean_loss,var_loss
        for rssi_gpr, rssi_gt, pos_gt in train_ds:
            
#             print(rssi_gpr[0][0],rssi_gt[0][0],pos_gt[0][0])
#             raise
            vae_loss, vae_l1_loss,vae_l1_loss_h,mloss,vloss,kl_loss = train_step(rssi_gpr, rssi_gt, pos_gt,True)
            vloss_li.append(vloss.numpy())
            mloss_li.append(mloss.numpy())
            vae_loss_li.append(vae_loss.numpy())
            vae_loss_li_l1.append(vae_l1_loss.numpy())
            pos_loss_li.append(vae_l1_loss_h.numpy())
            kl_li.append(kl_loss.numpy())
#             pos_loss_li.append(pos_loss.numpy())
        tf.print(f'{time.time() - start_time: 06.0f}s Epoch: {epoch+1} vae_loss: {np.nanmean(vae_loss_li):.2f} vae_loss_l1: {np.nanmean(vae_loss_li_l1):.2f} vae_loss_l1_h: {np.nanmean(pos_loss_li):.2f} mloss: {np.nanmean(mloss_li):.2f} vloss: {np.nanmean(vloss_li):.2f}', end = ' ')
        tf.print(f'kl_loss {np.mean(kl_li):.2f}')
#         print()
        if np.nanmean(vae_loss_li) <= 80 and flag == 0:
            print('lr:1e-5')
            vae_optimizer = tf.keras.optimizers.Adam(1e-5)
            flag = 1
            
        if np.nanmean(vae_loss_li) <= 50 and flag == 1:
            print('lr:1e-6')
            vae_optimizer = tf.keras.optimizers.Adam(1e-6)
            flag = 2

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
# lr_decayed_fn = tf.compat.v1.train.cosine_decay(1e-3, 1000,1)

z_dim = int(AP*0.8)
initializer = tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.01, seed=None
)

vae = VAE(z_dim,initializer)

In [16]:
vae_optimizer = tf.keras.optimizers.Adam(1e-4)

## 開始訓練

In [17]:
# min_l1,min_l1_ep = 
trainer(0, 10) #NO標準化 D_5G

 00005s Epoch: 1 vae_loss: 11248.05 vae_loss_l1: 81.40 vae_loss_l1_h: 42.14 mloss: 7006.24 vloss: 21630.38 kl_loss 0.00
 00009s Epoch: 2 vae_loss: 11247.54 vae_loss_l1: 81.39 vae_loss_l1_h: 42.14 mloss: 7005.93 vloss: 21630.38 kl_loss 0.00
 00015s Epoch: 3 vae_loss: 11244.26 vae_loss_l1: 81.37 vae_loss_l1_h: 42.11 mloss: 7003.87 vloss: 21630.38 kl_loss 0.10
 00019s Epoch: 4 vae_loss: 11058.37 vae_loss_l1: 78.81 vae_loss_l1_h: 39.68 mloss: 6760.76 vloss: 21608.08 kl_loss 149.95
 00022s Epoch: 5 vae_loss: 6427.66 vae_loss_l1: 31.58 vae_loss_l1_h: 18.03 mloss: 2445.08 vloss: 16483.69 kl_loss 910.87
 00025s Epoch: 6 vae_loss: 3257.25 vae_loss_l1: 16.03 vae_loss_l1_h: 21.19 mloss: 847.20 vloss: 14776.08 kl_loss 259.68
 00028s Epoch: 7 vae_loss: 2925.29 vae_loss_l1: 20.68 vae_loss_l1_h: 20.25 mloss: 1217.12 vloss: 22160.94 kl_loss 191.59
 00030s Epoch: 8 vae_loss: 2216.92 vae_loss_l1: 21.08 vae_loss_l1_h: 21.36 mloss: 1489.70 vloss: 37912.77 kl_loss 208.85
 00032s Epoch: 9 vae_loss: 1791.26 

## 設置 expand dataloader

In [18]:
ext_inter_data = np.load(path+'/RSSI_pred_grid.npy', allow_pickle=True)
ext_pos_data = np.load(path+'/grid_POS.npy', allow_pickle=True)
ext_real_data = np.load(path+'/RSSI_pred_grid.npy', allow_pickle=True)

def grid_generator():
    li = []
    length_t = len(ext_inter_data)
    for index in range(length_t):
        real_array = np.array(ext_real_data[index], dtype=np.float32).reshape(AP,1)
        inter_array = np.array(ext_inter_data[index], dtype=np.float32).reshape(AP,1)
        pos_array = np.array(ext_pos_data[index], dtype=np.float32)
        
        yield inter_array, real_array, pos_array

## 產生外擴資料

In [19]:
batch_size = 10000
grid_ds = tf.data.Dataset.from_generator(
    grid_generator, 
    output_shapes=((AP,1),(AP,1),(2,)), 
    output_types=(tf.float32,tf.float32,tf.float32))

AUTOTUNE = tf.data.experimental.AUTOTUNE
grid_ds = grid_ds.batch(batch_size).cache().prefetch(buffer_size=AUTOTUNE)#.repeat(10)#.shuffle(BUFFER_SIZE)

In [20]:
for i,(rssi_gpr, rssi_gt, pos_gt) in enumerate(grid_ds):
    reconstruction, mu, log_var, z = vae(rssi_gpr)
    if i == 0:
        train_pre = reconstruction.numpy()
    else:
        train_pre = np.concatenate((train_pre,reconstruction.numpy()),axis = 0)

np.save(os.path.join(path, 'pre_grid.npy'), train_pre)

In [21]:
train_pre.shape,path

((690, 158), './GPR_npy/npy_train_data')

## 儲存 model weights

In [22]:
print(path)
vae.save_weights('./mean_var_b3_AP'+str(AP)+'_'+str(z_dim)+'/savepoint', save_format='tf')
# test.load_weights('test_b3_drop')

./GPR_npy/npy_train_data
